# TODO
- Understand code
- understand how to extend to recognize sortcode and account number as entities

In [2]:
# download presidio
!pip install presidio_analyzer presidio_anonymizer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 920.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.5/278.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


Path to notebook: [https://www.github.com/microsoft/presidio/blob/main/docs/samples/python/pseudonomyzation.ipynb](https://www.github.com/microsoft/presidio/blob/main/docs/samples/python/pseudonomyzation.ipynb)

# Use Presidio Anonymizer for Pseudonymization of PII data

Pseudonymization is a data management and de-identification procedure by which personally identifiable information fields within a data record are replaced by one or more artificial identifiers, or pseudonyms. (https://en.wikipedia.org/wiki/Pseudonymization)

In this notebook, we'll show an example of how to use the Presidio Anonymizer library to pseudonymize PII data. In this example, we will replace each value with a unique identifier (e.g. <PERSON_14>). Then, we'll de-anonymize the data by replacing the unique identifiers back with their mapped PII values.

#### **Important**: The following logic is *not thread-safe* and may produce incorrect results if run concurrently in a multi-threaded environment, since the mapping has to be shared between threads/workers/processes.


In [3]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine, DeanonymizeEngine, OperatorConfig
from presidio_anonymizer.operators import Operator, OperatorType

from typing import Dict
from pprint import pprint

#### 1. Using the `AnalyzerEngine` to identify PII in a text

In [4]:
text = "Peter gave his book to Heidi which later gave it to Nicole. Peter lives in London and Nicole lives in Tashkent."
print("original text:")
pprint(text)
analyzer = AnalyzerEngine()
analyzer_results = analyzer.analyze(text=text, language="en")
print("analyzer results:")
pprint(analyzer_results)


original text:
('Peter gave his book to Heidi which later gave it to Nicole. Peter lives in '
 'London and Nicole lives in Tashkent.')
analyzer results:
[type: PERSON, start: 0, end: 5, score: 0.85,
 type: PERSON, start: 23, end: 28, score: 0.85,
 type: PERSON, start: 52, end: 58, score: 0.85,
 type: PERSON, start: 60, end: 65, score: 0.85,
 type: LOCATION, start: 75, end: 81, score: 0.85,
 type: PERSON, start: 86, end: 92, score: 0.85,
 type: LOCATION, start: 102, end: 110, score: 0.85]


#### 2. Creating a custom Anonymizer (called Operator) which replaces each text with a unique identifier.

To create a custom anonymizer, we need to create a class that inherits from `Operator` and implement the `operate` method. This method receives the original text and a dictionary called `params` with the configuration defined by the user. The method should return the anonymized text.

In this example we also implement the `validate` method to check that the input parameters are available, i.e. that the `entity_type` and `entity_mapping` parameters are defined, as they are required for this specific anonymizer. `entity_mapping` is a dictionary that maps each entity value to a unique identifier, for each entity type.

In [5]:
class InstanceCounterAnonymizer(Operator):
    """
    Anonymizer which replaces the entity value
    with an instance counter per entity.
    """

    REPLACING_FORMAT = "<{entity_type}_{index}>"

    def operate(self, text: str, params: Dict = None) -> str:
        """Anonymize the input text."""

        entity_type: str = params["entity_type"]

        # entity_mapping is a dict of dicts containing mappings per entity type
        entity_mapping: Dict[Dict:str] = params["entity_mapping"]

        entity_mapping_for_type = entity_mapping.get(entity_type)
        if not entity_mapping_for_type:
            new_text = self.REPLACING_FORMAT.format(
                entity_type=entity_type, index=0
            )
            entity_mapping[entity_type] = {}

        else:
            if text in entity_mapping_for_type:
                return entity_mapping_for_type[text]

            previous_index = self._get_last_index(entity_mapping_for_type)
            new_text = self.REPLACING_FORMAT.format(
                entity_type=entity_type, index=previous_index + 1
            )

        entity_mapping[entity_type][text] = new_text
        return new_text

    @staticmethod
    def _get_last_index(entity_mapping_for_type: Dict) -> int:
        """Get the last index for a given entity type."""

        def get_index(value: str) -> int:
            return int(value.split("_")[-1][:-1])

        indices = [get_index(v) for v in entity_mapping_for_type.values()]
        return max(indices)

    def validate(self, params: Dict = None) -> None:
        """Validate operator parameters."""

        if "entity_mapping" not in params:
            raise ValueError("An input Dict called `entity_mapping` is required.")
        if "entity_type" not in params:
            raise ValueError("An entity_type param is required.")

    def operator_name(self) -> str:
        return "entity_counter"

    def operator_type(self) -> OperatorType:
        return OperatorType.Anonymize

#### 3. Passing the new operator to the `AnonymizerEngine` and use it to anonymize the text.

In [6]:
# Create Anonymizer engine and add the custom anonymizer
anonymizer_engine = AnonymizerEngine()
anonymizer_engine.add_anonymizer(InstanceCounterAnonymizer)

# Create a mapping between entity types and counters
entity_mapping = dict()

# Anonymize the text

anonymized_result = anonymizer_engine.anonymize(
    text,
    analyzer_results,
    {
        "DEFAULT": OperatorConfig(
            "entity_counter", {"entity_mapping": entity_mapping}
        )
    },
)

print(anonymized_result.text)


<PERSON_1> gave his book to <PERSON_2> which later gave it to <PERSON_0>. <PERSON_1> lives in <LOCATION_1> and <PERSON_0> lives in <LOCATION_0>.


Note that the order is reversed due to the way entities are replaced in Presidio.

Since the user/client is holding the entity_mapping, it is possible to use it for de-anonymization as well. First, let's look at its contents.

In [7]:
pprint(entity_mapping, indent=2)

{ 'LOCATION': {'London': '<LOCATION_1>', 'Tashkent': '<LOCATION_0>'},
  'PERSON': { 'Heidi': '<PERSON_2>',
              'Nicole': '<PERSON_0>',
              'Peter': '<PERSON_1>'}}


#### 4. De-anonymizing the text using the entity_mapping

Similar to the anonymization operator, we need to create a custom de-anonymization operator. This operator will replace the unique identifiers with the original values.

In [8]:
class InstanceCounterDeanonymizer(Operator):
    """
    Deanonymizer which replaces the unique identifier 
    with the original text.
    """

    def operate(self, text: str, params: Dict = None) -> str:
        """Anonymize the input text."""

        entity_type: str = params["entity_type"]

        # entity_mapping is a dict of dicts containing mappings per entity type
        entity_mapping: Dict[Dict:str] = params["entity_mapping"]

        if entity_type not in entity_mapping:
            raise ValueError(f"Entity type {entity_type} not found in entity mapping!")
        if text not in entity_mapping[entity_type].values():
            raise ValueError(f"Text {text} not found in entity mapping for entity type {entity_type}!")

        return self._find_key_by_value(entity_mapping[entity_type], text)

    @staticmethod
    def _find_key_by_value(entity_mapping, value):
        for key, val in entity_mapping.items():
            if val == value:
                return key
        return None
    
    def validate(self, params: Dict = None) -> None:
        """Validate operator parameters."""

        if "entity_mapping" not in params:
            raise ValueError("An input Dict called `entity_mapping` is required.")
        if "entity_type" not in params:
            raise ValueError("An entity_type param is required.")

    def operator_name(self) -> str:
        return "entity_counter_deanonymizer"

    def operator_type(self) -> OperatorType:
        return OperatorType.Deanonymize


In [9]:
deanonymizer_engine = DeanonymizeEngine()
deanonymizer_engine.add_deanonymizer(InstanceCounterDeanonymizer)

deanonymized = deanonymizer_engine.deanonymize(
    anonymized_result.text, 
    anonymized_result.items, 
    {"DEFAULT": OperatorConfig("entity_counter_deanonymizer", 
                               params={"entity_mapping": entity_mapping})}
)
print("anonymized text:")
pprint(anonymized_result.text)
print("de-anonymized text:")
pprint(deanonymized.text)

anonymized text:
('<PERSON_1> gave his book to <PERSON_2> which later gave it to <PERSON_0>. '
 '<PERSON_1> lives in <LOCATION_1> and <PERSON_0> lives in <LOCATION_0>.')
de-anonymized text:
('Peter gave his book to Heidi which later gave it to Nicole. Peter lives in '
 'London and Nicole lives in Tashkent.')


## Adding a Sort Code recognizer
- We want to anonymize sort codes in the format dd-dd-dd

In [11]:
from typing import List
import pprint

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

In [18]:
# Define the regex pattern in a Presidio `Pattern` object:
# sortcode_pattern = Pattern(name="sortcode_pattern",regex=r"\d{2}-\d{2}-\d{2}", score = 0.5)
sortcode_pattern = Pattern(name="sortcode_pattern",regex=r"(?:sort code\s*)?(\d{2}[ -]?\d{2}[ -]?\d{2})", score = 0.5)

# Define the recognizer with one or more patterns
sortcode_recognizer = PatternRecognizer(supported_entity="SORTCODE", patterns = [sortcode_pattern])

In [19]:
text2 = "My sort code is 20-49-17. I have another account where the sort code is different, it's 45-09-34. There is another one where its 12 45 23. The final sort code 768643"

sortcode_result = sortcode_recognizer.analyze(text=text2, entities=["SORTCODE"])
print("Result:")
print(sortcode_result)

Result:
[type: SORTCODE, start: 16, end: 24, score: 0.5, type: SORTCODE, start: 88, end: 96, score: 0.5, type: SORTCODE, start: 129, end: 137, score: 0.5, type: SORTCODE, start: 149, end: 165, score: 0.5]


In [20]:
analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(sortcode_recognizer)

sortcode_result2 = analyzer.analyze(text=text2, language="en")
print("Results:")
print("\n".join([str(res) for res in sortcode_result2]))

Results:
type: DATE_TIME, start: 88, end: 96, score: 0.85
type: SORTCODE, start: 16, end: 24, score: 0.5
type: SORTCODE, start: 88, end: 96, score: 0.5
type: SORTCODE, start: 129, end: 137, score: 0.5
type: SORTCODE, start: 149, end: 165, score: 0.5
type: US_DRIVER_LICENSE, start: 159, end: 165, score: 0.01


In [21]:
anonymizer = AnonymizerEngine()

anonymized_text = anonymizer.anonymize(text=text2, analyzer_results=sortcode_result2)

print(anonymized_text)

text: My sort code is <SORTCODE>. I have another account where the sort code is different, it's <DATE_TIME>. There is another one where its <SORTCODE>. The final <SORTCODE>
items:
[
    {'start': 156, 'end': 166, 'entity_type': 'SORTCODE', 'text': '<SORTCODE>', 'operator': 'replace'},
    {'start': 134, 'end': 144, 'entity_type': 'SORTCODE', 'text': '<SORTCODE>', 'operator': 'replace'},
    {'start': 90, 'end': 101, 'entity_type': 'DATE_TIME', 'text': '<DATE_TIME>', 'operator': 'replace'},
    {'start': 16, 'end': 26, 'entity_type': 'SORTCODE', 'text': '<SORTCODE>', 'operator': 'replace'}
]



In [ ]:
from presidio_analyzer import Pattern, PatternRecognizer


class SortCodeRecognizer(PatternRecognizer):
    """
    Recognizes SortCode numbers using regex.

    :param patterns: List of patterns to be used by this recognizer
    :param context: List of context words to increase confidence in detection
    :param supported_language: Language this recognizer supports
    :param supported_entity: The entity this recognizer can detect
    """

    # Weak pattern: all passport numbers are a weak match, e.g., 14019033
    PATTERNS = [
        Pattern("Sort Code (very weak)", r"(\b[0-9]{9}\b)", 0.05),
        Pattern("Sort Code (Medium)", r"(\b[A-Z][0-9]{8}\b)", 0.1),
        Pattern("Sort Code (Hard)", r"(\b[A-Z][0-9]{8}\b)", 0.8),
    ]
    CONTEXT = ["sort code", "routing number"]

    def __init__(
        self,
        patterns: Optional[List[Pattern]] = None,
        context: Optional[List[str]] = None,
        supported_language: str = "en",
        supported_entity: str = "SORTCODE",
    ):
        patterns = patterns if patterns else self.PATTERNS
        context = context if context else self.CONTEXT
        super().__init__(
            supported_entity=supported_entity,
            patterns=patterns,
            context=context,
            supported_language=supported_language,
        )